# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_excertos.CSV', sep = ';')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_EXCERTO,TEXTO_EXCERTO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995438,Voto:Cuidam os autos de tomada de contas espec...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025603,Voto:Cuidam os autos de Solicitação do Congres...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455375,Relatório:Trata-se de embargos de declaração o...,Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773747,Voto:8. Em relação a outros processos judiciai...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773403,Voto:11. Relativamente ao ato envolvendo a Sra...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13285, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          553
Contrato Administrativo     941
Convênio                    683
Desestatização              139
Direito Processual         1811
Finanças Públicas           328
Gestão Administrativa       338
Licitação                  2756
Pessoal                    3393
Responsabilidade           2343
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13285, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 63925
limite_texto = 2000
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['TEXTO_EXCERTO'])

sequences = tokenizer.texts_to_sequences(df['TEXTO_EXCERTO'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 63925 unique tokens.


In [8]:
max = 0
lens = []
for seq in sequences:
    lens.append(len(seq))
np.min(lens), np.mean(lens), np.max(lens), np.std(lens), len(lens), sum(pd.Series(lens) > 2000)

(34, 604.8516371847949, 5324, 506.4573284699563, 13285, 297)

In [9]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13285, 2000)


In [10]:
x.shape, y.shape

((13285, 2000), (13285, 10))

# Treinamento

In [11]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(SimpleRNN(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W1102 01:16:28.598615 140137225324352 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1102 01:16:28.624496 140137225324352 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1102 01:16:28.630220 140137225324352 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1102 01:16:28.823641 140137225324352 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               78592     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 3,277,412
Trainable params: 3,277,412
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1102 01:16:29.074687 140137225324352 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1102 01:16:29.262429 140137225324352 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 252s 24ms/step - loss: 1.9922 - categorical_accuracy: 0.2649 - val_loss: 2.5651 - val_categorical_accuracy: 0.0613
Epoch 2/20
10628/10628 [==============================] - 242s 23ms/step - loss: 1.8122 - categorical_accuracy: 0.3694 - val_loss: 2.7097 - val_categorical_accuracy: 0.1031
Epoch 3/20
10628/10628 [==============================] - 228s 21ms/step - loss: 1.7264 - categorical_accuracy: 0.4141 - val_loss: 2.0119 - val_categorical_accuracy: 0.3481
Epoch 4/20
10628/10628 [==============================] - 227s 21ms/step - loss: 1.6599 - categorical_accuracy: 0.4387 - val_loss: 2.4508 - val_categorical_accuracy: 0.2548
Epoch 5/20
10628/10628 [==============================] - 227s 21ms/step - loss: 1.6076 - categorical_accuracy: 0.4586 - val_loss: 1.7873 - val_categorical_accuracy: 0.3880
Epoch 6/20
10628/10628 [==============================] - 225s 21ms/step - loss: 1.547

In [14]:
from keras.layers import GRU

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 3,434,596
Trainable params: 3,434,596
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 536s 50ms/step - loss: 1.6401 - categorical_accuracy: 0.4453 - val_loss: 1.4779 - val_categorical_accuracy: 0.5085
Epoch 2/20
10628/10628 [==============================] - 535s 50ms/step - loss: 0.9916 - categorical_accuracy: 0.6727 - val_loss: 1.4180 - val_categorical_accuracy: 0.5213
E

In [15]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               314368    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 3,513,188
Trainable params: 3,513,188
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 683s 64ms/step - loss: 1.6178 - categorical_accuracy: 0.4539 - val_loss: 1.7710 - val_categorical_accuracy: 0.3688
Epoch 2/20
10628/10628 [==============================] - 682s 64ms/step - loss: 1.4517 - categorical_accuracy: 0.5404 - val_loss: 1.8619 - val_categorical_accuracy: 0.3767
E

In [16]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(SimpleRNN(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               157184    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 3,358,564
Trainable params: 3,358,564
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 331s 31ms/step - loss: 2.0790 - categorical_accuracy: 0.2394 - val_loss: 2.1842 - val_categorical_accuracy: 0.1701
Epoch 2/20
10628/10628 [==============================] - 331s 31ms/step - loss: 1.9136 - categorical_accuracy: 0.3236 - val_loss: 1.9024 - val_categorical_accuracy: 0.2096
E

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 3,672,932
Trainable params: 3,672,932
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1158s 109ms/step - loss: 1.5921 - categorical_accuracy: 0.4584 - val_loss: 1.5148 - val_categorical_accuracy: 0.4780
Epoch 2/20
10628/10628 [==============================] - 1234s 116ms/step - loss: 1.2323 - categorical_accuracy: 0.6053 - val_loss: 1.2828 - val_categorical_accuracy: 0.55

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               628736    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 3,830,116
Trainable params: 3,830,116
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1266s 119ms/step - loss: 1.5400 - categorical_accuracy: 0.4843 - val_loss: 1.5529 - val_categorical_accuracy: 0.4753
Epoch 2/20
10628/10628 [==============================] - 1260s 119ms/step - loss: 1.9613 - categorical_accuracy: 0.3182 - val_loss: 2.0051 - val_categorical_accuracy: 0.21

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(128))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
gru_3 (GRU)                  (None, 128)               68736     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 3,266,276
Trainable params: 3,266,276
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 682s 64ms/step - loss: 1.5792 - categorical_accuracy: 0.4629 - val_loss: 1.5595 - val_categorical_accuracy: 0.4791
Epoch 2/20
10628/10628 [==============================] - 690s 65ms/step - loss: 1.0020 - categorical_accuracy: 0.6663 - val_loss: 1.2288 - val_categorical_accuracy: 0.5709
E

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(64))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
gru_4 (GRU)                  (None, 64)                22080     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                650       
Total params: 3,218,980
Trainable params: 3,218,980
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 548s 52ms/step - loss: 1.6664 - categorical_accuracy: 0.4186 - val_loss: 1.4773 - val_categorical_accuracy: 0.5254
Epoch 2/20
10628/10628 [==============================] - 547s 51ms/step - loss: 1.1673 - categorical_accuracy: 0.6170 - val_loss: 1.3490 - val_categorical_accuracy: 0.5393
E

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 2000, 50)          3196250   
_________________________________________________________________
gru_5 (GRU)                  (None, 512)               864768    
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 4,066,148
Trainable params: 4,066,148
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 542s 51ms/step - loss: 1.6309 - categorical_accuracy: 0.4481 - val_loss: 1.6788 - val_categorical_accuracy: 0.4257
Epoch 2/20
10628/10628 [==============================] - 540s 51ms/step - loss: 1.1163 - categorical_accuracy: 0.6350 - val_loss: 1.2987 - val_categorical_accuracy: 0.5668
E

In [22]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_6 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1290      
Total params: 3,466,212
Trainable params: 3,466,212
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 546s 51ms/step - loss: 1.6156 - categorical_accuracy: 0.4498 - val_loss: 1.5282 - val_categorical_accuracy: 0.5359
Epoch 2/20
10628/10628 [==================

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_12 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_13 (Dense)             (None, 10)                650       
Total params: 3,449,124
Trainable params: 3,449,124
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 538s 51ms/step - loss: 1.7292 - categorical_accuracy: 0.4208 - val_loss: 1.5714 - val_categorical_accuracy: 0.5021
Epoch 2/20
10628/10628 [==================

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_8 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_14 (Dense)             (None, 32)                8224      
_________________________________________________________________
dense_15 (Dense)             (None, 10)                330       
Total params: 3,440,580
Trainable params: 3,440,580
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 552s 52ms/step - loss: 1.6166 - categorical_accuracy: 0.4467 - val_loss: 1.3882 - val_categorical_accuracy: 0.5288
Epoch 2/20
10628/10628 [==================

In [25]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1103 19:08:17.200716 140137225324352 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_9 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_16 (Dense)             (None, 10)                2570      
Total params: 3,434,596
Trainable params: 3,434,596
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 668s 63ms/step - loss: 1.6731 - categorical_accuracy: 0.4275 - val_loss: 1.5606 - val_categorical_accuracy: 0.4930
Epoch 2/20
10628/10628 [==============================] - 665s 63ms/step - loss: 1.1040 - categorical_accuracy: 0.6385 - val_loss: 1.2987 - val_categorical_accuracy: 0.5969
E

In [26]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_10 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_17 (Dense)             (None, 10)                2570      
Total params: 3,434,596
Trainable params: 3,434,596
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 651s 61ms/step - loss: 1.6617 - categorical_accuracy: 0.4363 - val_loss: 1.4697 - val_categorical_accuracy: 0.5205
Epoch 2/20
10628/10628 [==============================] - 649s 61ms/step - loss: 1.1382 - categorical_accuracy: 0.6315 - val_loss: 1.2814 - val_categorical_accuracy: 0.5905
E

In [27]:
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_11 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_18 (Dense)             (None, 10)                2570      
Total params: 3,434,596
Trainable params: 3,434,596
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 542s 51ms/step - loss: 1.6931 - categorical_accuracy: 0.4411 - val_loss: 1.5485 - val_categorical_accuracy: 0.4445
Epoch 2/20
10628/10628 [==============================] - 540s 51ms/step - loss: 1.2781 - categorical_accuracy: 0.5925 - val_loss: 1.3842 - val_categorical_accuracy: 0.5574
E

In [28]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_12 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                2570      
Total params: 3,434,596
Trainable params: 3,434,596
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 556s 52ms/step - loss: 1.8082 - categorical_accuracy: 0.3659 - val_loss: 1.7801 - val_categorical_accuracy: 0.3320
Epoch 2/20
10628/10628 [==============================] - 553s 52ms/step - loss: 1.4038 - categorical_accuracy: 0.5467 - val_loss: 1.4684 - val_categorical_accuracy: 0.5017
E

In [29]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, return_sequences=True))
model.add(GRU(32))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_13 (GRU)                 (None, 2000, 256)         235776    
_________________________________________________________________
gru_14 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_20 (Dense)             (None, 10)                330       
Total params: 3,460,100
Trainable params: 3,460,100
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1126s 106ms/step - loss: 1.9252 - categorical_accuracy: 0.3317 - val_loss: 2.0354 - val_categorical_accuracy: 0.1788
Epoch 2/20
10628/10628 [================

In [30]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_15 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_21 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 10)                1290      
Total params: 3,466,212
Trainable params: 3,466,212
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 654s 62ms/step - loss: 1.

In [31]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=64, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_16 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_23 (Dense)             (None, 10)                2570      
Total params: 3,434,596
Trainable params: 3,434,596
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 275s 26ms/step - loss: 1.8095 - categorical_accuracy: 0.3735 - val_loss: 1.7334 - val_categorical_accuracy: 0.4110
Epoch 2/20
10628/10628 [==============================] - 272s 26ms/step - loss: 1.1457 - categorical_accuracy: 0.6190 - val_loss: 1.2736 - val_categorical_accuracy: 0.5657
E

In [32]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_17 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_24 (Dense)             (None, 10)                2570      
Total params: 3,434,596
Trainable params: 3,434,596
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 667s 63ms/step - loss: 1.7256 - categorical_accuracy: 0.4136 - val_loss: 1.5093 - val_categorical_accuracy: 0.5190
Epoch 2/20
10628/10628 [==============================] - 653s 61ms/step - loss: 1.2897 - categorical_accuracy: 0.5855 - val_loss: 1.5773 - val_categorical_accuracy: 0.4923
E

In [33]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(512, dropout=0.1, recurrent_dropout=0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_18 (GRU)                 (None, 512)               864768    
_________________________________________________________________
dense_25 (Dense)             (None, 10)                5130      
Total params: 4,066,148
Trainable params: 4,066,148
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 657s 62ms/step - loss: 1.7157 - categorical_accuracy: 0.4300 - val_loss: 1.6222 - val_categorical_accuracy: 0.4520
Epoch 2/20
10628/10628 [==============================] - 654s 62ms/step - loss: 1.2762 - categorical_accuracy: 0.5884 - val_loss: 1.3849 - val_categorical_accuracy: 0.5612
E

In [34]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(1024, dropout=0.1, recurrent_dropout=0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
gru_19 (GRU)                 (None, 1024)              3302400   
_________________________________________________________________
dense_26 (Dense)             (None, 10)                10250     
Total params: 6,508,900
Trainable params: 6,508,900
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 713s 67ms/step - loss: 1.7996 - categorical_accuracy: 0.3954 - val_loss: 1.7893 - val_categorical_accuracy: 0.4061
Epoch 2/20
10628/10628 [==============================] - 710s 67ms/step - loss: 1.4238 - categorical_accuracy: 0.5454 - val_loss: 1.5244 - val_categorical_accuracy: 0.4953
E

In [35]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(256, dropout=0.1, recurrent_dropout=0.4)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 3,672,932
Trainable params: 3,672,932
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1279s 120ms/step - loss: 1.6493 - categorical_accuracy: 0.4558 - val_loss: 1.5757 - val_categorical_accuracy: 0.4328
Epoch 2/20
10628/10628 [==============================] - 1270s 119ms/step - loss: 1.2663 - categorical_accuracy: 0.5908 - val_loss: 1.3822 - val_categorical_accuracy: 0.52

In [36]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(512, dropout=0.1, recurrent_dropout=0.4)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 1024)              1729536   
_________________________________________________________________
dense_28 (Dense)             (None, 10)                10250     
Total params: 4,936,036
Trainable params: 4,936,036
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1184s 111ms/step - loss: 1.7219 - categorical_accuracy: 0.4329 - val_loss: 2.9659 - val_categorical_accuracy: 0.2676
Epoch 2/20
10628/10628 [==============================] - 1177s 111ms/step - loss: 1.3560 - categorical_accuracy: 0.5632 - val_loss: 1.4165 - val_categorical_accuracy: 0.54

In [37]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(1024, dropout=0.1, recurrent_dropout=0.4)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 2000, 50)          3196250   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 2048)              6604800   
_________________________________________________________________
dense_29 (Dense)             (None, 10)                20490     
Total params: 9,821,540
Trainable params: 9,821,540
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 2293s 216ms/step - loss: 1.8464 - categorical_accuracy: 0.4254 - val_loss: 1.6406 - val_categorical_accuracy: 0.4321
Epoch 2/20
10628/10628 [==============================] - 1452s 137ms/step - loss: 1.5563 - categorical_accuracy: 0.5060 - val_loss: 1.8079 - val_categorical_accuracy: 0.40